Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [4]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

67921/67921 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [6]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [8]:
# windowing function

def create_windows_np(data, window_size, horizon, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray): Time series data with one dimension.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    shuffle (bool): Shuffle the windows or not.

    Returns:
    tuple: A tuple containing the input-output pairs (windows, targets) as NumPy arrays.
    """

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y



In [9]:
past, future = (5, 2)

X_train, y_train = create_windows_np(train_data,
                                  window_size=past,
                                  horizon=2,
                                  shuffle=False)

X_test, y_test = create_windows_np(test_data,
                                  window_size=past,
                                  horizon=1,
                                  shuffle=False)

# Reshape X_train and X_test to be 3D arrays for the GRU layer
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [17]:
inputs = keras.layers.Input(shape=(past,future))
x = layers.GRU(units=32, return_sequences=True)(inputs)
x = layers.GRU(units=32)(x)
outputs = layers.Dense(units=1)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 5, 32)          │         3,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,825 (38.38 KB)

 Trainable params: 9,825 (38.38 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 133.2945 - mae: 10.7076 - val_loss: 73.9035 - val_mae: 7.6780
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 44.2988 - mae: 5.6003 - val_loss: 35.2502 - val_mae: 4.8822
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 25.6808 - mae: 4.0431 - val_loss: 26.3014 - val_mae: 4.1192
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 20.4532 - mae: 3.5625 - val_loss: 21.9803 - val_mae: 3.7468
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 17.8122 - mae: 3.3439 - val_loss: 19.5988 - val_mae: 3.5488
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 17.1483 - mae: 3.3053 - val_loss: 18.2692 - val_mae: 3.4386
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 16.2627 - mae: 3.2414 - val_loss: 17.4605 - val_mae: 3.3600
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 15.2048 - mae: 3.0852 - val_loss: 16.1306 - val_mae: 3.1603
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7m

In [19]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.4019 - mae: 1.8328
Test Loss: [5.607117176055908, 1.8466960191726685]


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [10]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

Voy a redefinir la funcion de creacion de ventana para que pueda trabajar con mas
de una variable predictora. Lo mejor seria sobrescribir la funcion create_windows_np
pero para que las respuestas queden ordenadas voy a definir una nueva fucion create_windows_np_multi

In [11]:
## Create windows
def create_windows_np_multi(data, window_size, horizon, target_column_index, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy with multiple features.

    Parameters:
    data (np.ndarray): Time series data with multiple dimensions (features).
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict for the target.
    target_column_index (int): The index of the column in 'data' that is the target variable.
    shuffle (bool): Shuffle the windows or not.

    Returns:
    tuple: A tuple containing the input-output pairs (windows, targets) as NumPy arrays.
    """

    X, y = [], []
    n_features = data.shape[1] # Obtiene el número de features

    for i in range(len(data) - window_size - horizon + 1):
        # X ahora contiene las ventanas de todas las features
        X.append(data[i:i+window_size, :])
        # y sigue siendo el valor futuro de la columna objetivo
        y.append(data[i+window_size+horizon-1, target_column_index])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

target_column_index = df_multi.columns.get_loc('Temp')

X_train, y_train = create_windows_np_multi(train_data.values,
                                           window_size=past,
                                           horizon=future,
                                           target_column_index=target_column_index,
                                           shuffle=False)

X_test, y_test = create_windows_np_multi(test_data.values,
                                         window_size=past,
                                         horizon=future,
                                         target_column_index=target_column_index,
                                         shuffle=False)

In [13]:
n_features = X_train.shape[2]

inputs = keras.layers.Input(shape=(past, n_features))

x = layers.GRU(units=32, return_sequences=True)(inputs)
x = layers.GRU(units=32)(x)
outputs = layers.Dense(units=1)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 5, 32)          │         3,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,825 (38.38 KB)

 Trainable params: 9,825 (38.38 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 104.9281 - mae: 9.3308 - val_loss: 59.1237 - val_mae: 6.6887
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 39.7289 - mae: 5.2055 - val_loss: 35.5779 - val_mae: 4.9104
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 25.4407 - mae: 4.0306 - val_loss: 26.9518 - val_mae: 4.1744
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 20.6116 - mae: 3.5858 - val_loss: 22.3807 - val_mae: 3.7809
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 18.0980 - mae: 3.3623 - val_loss: 19.8718 - val_mae: 3.5712
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 17.5556 - mae: 3.3511 - val_loss: 18.3744 - val_mae: 3.4480
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 16.2867 - mae: 3.1986 - val_loss: 17.5306 - val_mae: 3.3696
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 15.7349 - mae: 3.1472 - val_loss: 16.4623 - val_mae: 3.1997
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/

In [15]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.7243 - mae: 2.0420
Test Loss: [6.85291051864624, 2.0490520000457764]


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


a,b,d

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.
